In [98]:
import pandas as pd
import numpy as np
import seaborn as sns

In [99]:
df = pd.read_csv("train.csv")
df.info()
df.describe()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215258 entries, 0 to 215257
Columns: 122 entries, ID to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 200.4+ MB


,ID,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,196348,0,Cash loans,F,N,Y,2,225000.0,785398.5,33403.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
1,147976,0,Cash loans,M,N,Y,0,90000.0,592560.0,35937.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,52662,0,Revolving loans,F,N,Y,1,126000.0,202500.0,10125.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
3,101577,0,Cash loans,F,N,Y,0,157500.0,254700.0,18531.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,173078,0,Cash loans,F,Y,N,0,540000.0,1252363.5,47830.5,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,5.0


In [100]:
df['TARGET'].value_counts()

TARGET
0    197880
1     17378
Name: count, dtype: int64

 XỬ LÝ MISSING VALUES 

In [101]:
# Xác định các cột số và cột danh mục
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
cat_cols = df.select_dtypes(include=["object"]).columns

print("Cột số:", len(num_cols), ":", num_cols)
print("Cột danh mục:", len(cat_cols), ":", cat_cols)

Cột số: 106 : Index(['ID', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE',
       'DAYS_BIRTH', 'DAYS_EMPLOYED',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=106)
Cột danh mục: 16 : Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')


In [102]:
# Kiểm tra missing values
print("Tổng số cột có missing values:", df.isnull().any().sum())

missing_counts = df.isnull().sum().sort_values(ascending=False)
print("\nTop cột có nhiều giá trị missing nhất (số lượng):")
print(missing_counts.head(10))

missing_perc = (df.isnull().mean()*100).sort_values(ascending=False)
print("\nTop cột có nhiều giá trị missing nhất (phần trăm):")
print(missing_perc.head(10))

Tổng số cột có missing values: 67

Top cột có nhiều giá trị missing nhất (số lượng):
COMMONAREA_MODE             150500
COMMONAREA_AVG              150500
COMMONAREA_MEDI             150500
NONLIVINGAPARTMENTS_MODE    149455
NONLIVINGAPARTMENTS_AVG     149455
NONLIVINGAPARTMENTS_MEDI    149455
FONDKAPREMONT_MODE          147218
LIVINGAPARTMENTS_MEDI       147175
LIVINGAPARTMENTS_AVG        147175
LIVINGAPARTMENTS_MODE       147175
dtype: int64

Top cột có nhiều giá trị missing nhất (phần trăm):
COMMONAREA_MODE             69.916101
COMMONAREA_AVG              69.916101
COMMONAREA_MEDI             69.916101
NONLIVINGAPARTMENTS_MODE    69.430637
NONLIVINGAPARTMENTS_AVG     69.430637
NONLIVINGAPARTMENTS_MEDI    69.430637
FONDKAPREMONT_MODE          68.391419
LIVINGAPARTMENTS_MEDI       68.371443
LIVINGAPARTMENTS_AVG        68.371443
LIVINGAPARTMENTS_MODE       68.371443
dtype: float64

Top cột có nhiều giá trị missing nhất (phần trăm):
COMMONAREA_MODE             69.916101
COMMONAREA_AVG 

In [103]:
# Numeric → thay median
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Categorical → thay mode
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])
    
print("Giá trị missing sau khi xử lý:", df.isnull().any().sum())

Giá trị missing sau khi xử lý: 0


XỬ LÝ OULIERS

In [104]:
# Check outliers bằng phương pháp IQR
outlier_stats = {}
total_rows = len(df)

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_count = len(outliers)
    outlier_percent = 100 * outlier_count / total_rows
    outlier_stats[col] = (outlier_count, outlier_percent)
print("Tổng số outliers trước xử lý:", sum(count for count, _ in outlier_stats.values()))
# Tổng số cột có outliers
cols_with_outliers = [col for col, (count, _) in outlier_stats.items() if count > 0]
print(f"Tổng số cột có outliers: {len(cols_with_outliers)} trên {len(num_cols)} cột số ({100*len(cols_with_outliers)/len(num_cols):.2f}%)")

# Top 5 cột có nhiều outliers nhất
top_outlier_cols = sorted(outlier_stats.items(), key=lambda x: x[1][0], reverse=True)[:5]
print("Top 5 cột có nhiều outliers nhất:")
for col, (count, percent) in top_outlier_cols:
    print(f"- {col}: {count} outliers ({percent:.2f}% của cột)")

Tổng số outliers trước xử lý: 4033394
Tổng số cột có outliers: 100 trên 106 cột số (94.34%)
Top 5 cột có nhiều outliers nhất:
- LIVINGAREA_MEDI: 107109 outliers (49.76% của cột)
- LIVINGAREA_AVG: 107107 outliers (49.76% của cột)
- LIVINGAREA_MODE: 107000 outliers (49.71% của cột)
- APARTMENTS_AVG: 105597 outliers (49.06% của cột)
- APARTMENTS_MEDI: 105444 outliers (48.98% của cột)


Khi kiểm tra IQR, hơn **90% các cột** bị phát hiện có outliers.  
Vì phần lớn biến trong dataset có **phân phối lệch mạnh (skewed)**, IQR sẽ báo *quá nhiều* outliers → không khả thi.

In [105]:
# Xử lý Outliers bằng Percentile Clipping

def clip_percentile(col, lower=0.01, upper=0.99):
    low = col.quantile(lower)
    high = col.quantile(upper)
    return col.clip(lower=low, upper=high)

# Áp dụng cho numeric cột có outliers (trừ khi là binary)
numeric_for_clip = [c for c in num_cols if df[c].nunique() > 2]

for col in numeric_for_clip:
    df[col] = clip_percentile(df[col])


Thay vì loại bỏ dòng, ta sẽ:
- Cắt (clip) các giá trị **dưới phân vị 1%**
- Và giá trị **trên phân vị 99%**

 Ưu điểm:
- Không mất dữ liệu
- Giảm ảnh hưởng của extreme values
- Giữ được phân phối của biến
- Rất phù hợp cho mô hình tree-based (XGBoost, LightGBM)

In [106]:
# Sau xử lý
outlier_stats_after = {}
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier_count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    outlier_stats_after[col] = outlier_count

print("Tổng số outliers sau xử lý:", sum(outlier_stats_after.values()))

Tổng số outliers sau xử lý: 4028870


ENCODE CATEGORICAL VARIABLES

- Nếu cột chỉ có **2 giá trị** → dùng **Label Encoding** (biến thành 0/1)
- Nếu cột có **nhiều hơn 2 giá trị** → dùng **One-Hot Encoding**

In [107]:
from sklearn.preprocessing import LabelEncoder, RobustScaler

for col in cat_cols:
    if df[col].nunique() == 2:
        df[col] = LabelEncoder().fit_transform(df[col])
    else:
        df = pd.get_dummies(df, columns=[col], drop_first=True)

In [108]:
print(df.dtypes)

ID                                 float64
TARGET                               int64
NAME_CONTRACT_TYPE                   int32
FLAG_OWN_CAR                         int32
FLAG_OWN_REALTY                      int32
                                    ...   
WALLSMATERIAL_MODE_Monolithic         bool
WALLSMATERIAL_MODE_Others             bool
WALLSMATERIAL_MODE_Panel              bool
WALLSMATERIAL_MODE_Stone, brick       bool
WALLSMATERIAL_MODE_Wooden             bool
Length: 230, dtype: object


SCALING DỮ LIỆU NUMERIC

RobustScaler tốt hơn Standard/MinMax vì:
- Dùng median
- Không bị tác động bởi outliers
- Tốt cho Logistic Regression / SVM / KNN
- Không phá vỡ performance của tree models

In [109]:
scaler = RobustScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

print("Kiểm tra dữ liệu numeric sau scaling:")
print(df[num_cols].describe())

Kiểm tra dữ liệu numeric sau scaling:
                 ID         TARGET   CNT_CHILDREN  AMT_INCOME_TOTAL  \
count  2.152580e+05  215258.000000  215258.000000     215258.000000   
mean   1.155311e-19       0.080731       0.414298          0.243848   
std    5.770122e-01       0.272422       0.707681          0.921802   
min   -9.800000e-01       0.000000       0.000000         -1.100000   
25%   -5.000000e-01       0.000000       0.000000         -0.350000   
50%    0.000000e+00       0.000000       0.000000          0.000000   
75%    5.000000e-01       0.000000       1.000000          0.650000   
max    9.800000e-01       1.000000       3.000000          3.650000   

          AMT_CREDIT    AMT_ANNUITY  AMT_GOODS_PRICE  \
count  215258.000000  215258.000000    215258.000000   
mean        0.154523       0.112277         0.197378   
std         0.727032       0.754759         0.820921   
min        -0.811512      -1.035280        -0.867347   
25%        -0.452114      -0.463851       

KIỂM TRA DỮ LIỆU SAU XỬ LÍ

In [110]:
print("Kiểm tra lại dữ liệu sau tất cả các bước xử lý:")
print(df.info())
print(df.describe())

Kiểm tra lại dữ liệu sau tất cả các bước xử lý:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215258 entries, 0 to 215257
Columns: 230 entries, ID to WALLSMATERIAL_MODE_Wooden
dtypes: bool(120), float64(106), int32(4)
memory usage: 202.0 MB
None
                 ID         TARGET  NAME_CONTRACT_TYPE   FLAG_OWN_CAR  \
count  2.152580e+05  215258.000000       215258.000000  215258.000000   
mean   1.155311e-19       0.080731            0.094761       0.339853   
std    5.770122e-01       0.272422            0.292885       0.473660   
min   -9.800000e-01       0.000000            0.000000       0.000000   
25%   -5.000000e-01       0.000000            0.000000       0.000000   
50%    0.000000e+00       0.000000            0.000000       0.000000   
75%    5.000000e-01       0.000000            0.000000       1.000000   
max    9.800000e-01       1.000000            1.000000       1.000000   

       FLAG_OWN_REALTY   CNT_CHILDREN  AMT_INCOME_TOTAL     AMT_CREDIT  \
count    215258.00